In [1]:
import os, sys, time, math
from natsort import natsorted, ns

import numpy as np
import cv2
import matplotlib.pyplot as plt
import cma
from PIL import Image
import glob
import re


from gvxrPython3 import gvxr
from gvxrPython3 import json2gvxr

from gvxrPython3.utils import visualise # Visualise the 3D environment if k3D is supported
from gvxrPython3.utils import plotScreenshot # Visualise the 3D environment using Matplotlib

from gvxrPython3.utils import loadSpekpySpectrum # Generate and load an X-ray spectrum using Spekpy
from gvxrPython3.utils import loadXpecgenSpectrum # Generate and load an X-ray spectrum using xpecgen

SimpleGVXR 2.0.6 (2023-05-23T20:52:16) [Compiler: Microsoft Visual Studio] on Windows
gVirtualXRay core library (gvxr) 2.0.6 (2023-05-23T20:53:11) [Compiler: Microsoft Visual Studio] on Windows


In [2]:
#data_path = "4thOCtober_image/"
data_path = "C:/Users/snn23kfl/project/"

In [3]:
current_folder = str(globals()['_dh'][0])
print(current_folder)

C:\Users\snn23kfl\project


In [4]:
def average_images(image_paths):
    
    """Average a list of images."""
    # Load the first image to get the shape
    sample_image = cv2.imread(image_paths[0], cv2.IMREAD_GRAYSCALE)
    if sample_image is None:
        raise ValueError(f"Failed to load image: {image_paths[0]}")
    
    avg_image = np.zeros_like(sample_image, dtype=float)
    
    for path in image_paths:
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError(f"Failed to load image: {path}")
        avg_image += image.astype(float)
    
    avg_image /= len(image_paths)
    
    return cv2.medianBlur(avg_image.astype(np.single), 3)

In [5]:
def flatField(img, white, dark, epsilon=0.0):

    temp_white = np.copy(white)
    temp_img = np.copy(img)
    
    test = white - dark == 0
    temp_white[test] += 1

    if len(img.shape) == 2:
        temp_img[test] += 1
        return (temp_img - dark + epsilon) / (temp_white - dark + epsilon)
    elif len(img.shape) == 3:
        flat = np.zeros(img.shape, dtype=np.single)
        for i, proj in enumerate(temp_img):
            proj[test] += 1
            flat[i] = (proj - dark + epsilon) / (temp_white - dark + epsilon)
        return flat
    else:
        raise IOError("Bad image dimension: " + str(img.shape))

In [6]:
#path for my recently acquired images stored in PhD file



dark_field_paths = glob.glob(data_path + '/darkfd/darkfd_*.tiff')
dark_field_paths = natsorted(dark_field_paths, key=lambda y: y.lower())

white_field_paths = glob.glob(data_path + '/whitefd/whitefd_*.tiff')
white_field_paths = natsorted(white_field_paths, key=lambda y: y.lower())

raw_image_paths = glob.glob(data_path + '/raw_images/raw_image_*.tiff')
raw_image_paths = natsorted(raw_image_paths, key=lambda y: y.lower())

if len(raw_image_paths) == 0:
    raw_image_paths = glob.glob(data_path + '/rawimages/raw_images_*.jpg')
    raw_image_paths = natsorted(raw_image_paths, key=lambda y: y.lower())

In [7]:
#Average the dark field and white field images
I_dark = average_images(dark_field_paths)
I_white = average_images(white_field_paths)

In [8]:
I_raw = []
for fname in raw_image_paths:
    I_raw.append(cv2.imread(fname,cv2.IMREAD_GRAYSCALE))
I_raw = np.array(I_raw, dtype=np.single)
#I_raw = average_images(raw_image_paths)

# for image in I_raw:
#     image /= np.mean(image[:,0:200])

# I_white /= np.mean(I_white[:,0:200])
# I_dark /= np.mean(I_dark[:,0:200])

I_flat1 = flatField(I_raw, I_white, I_dark)
I_flat2 = flatField(I_raw, I_white, np.zeros(I_dark.shape))

# Clamp pixel values
I_flat1[I_flat1<0] = 0
I_flat2[I_flat2<0] = 0

I_flat1[I_flat1>1] = 1
I_flat2[I_flat2>1] = 1

OSError: Bad image dimension: (0,)

In [ ]:
I_flat = I_flat2

In [ ]:
#sod = 43
#sod =47
sod = 48
sdd = 61

In [ ]:
Ti90Al10 = 5.68
x_src = 0
y_src = 0
z_src = sod

x_det = 0
y_det = 0
z_det = -(sdd - sod)

x_obj = 5
y_obj = 0
z_obj = 0

alpha_x = 0 
alpha_y = 0
alpha_z = 0

x_rot = 0
y_rot = 0
z_rot = 0

x_default = [
    x_src,
    y_src,
    z_src,

    x_det,
    y_det,
    z_det,

    x_obj,
    y_obj,
    z_obj,

    alpha_x, 
    alpha_y,
    alpha_z,

#     x_rot,
#     y_rot,
#     z_rot,
]


In [ ]:
# Initialise gVXR using our JSON file
json2gvxr.initGVXR("simulation1.json", "OPENGL", 3, 2)

In [ ]:
# Load our source properties
json2gvxr.initSourceGeometry("simulation1.json")
json2gvxr.initSpectrum(verbose=0)

gvxr.setSourcePosition(x_src, y_src, z_src, "mm")

In [ ]:
def getAverageEnergy(k, f):
    
    avg = 0
    for energy, count in zip(k, f):
        avg += energy * count
        
    return avg / np.sum(f)

In [ ]:
# Load our detector
json2gvxr.initDetector("simulation1.json")
    # "NumberOfPixels": [2880, 2880],

gvxr.setDetectorPosition(x_det, y_det, z_det, "mm")

In [ ]:
# Load our samples
json2gvxr.initSamples("simulation1.json", verbose=0)

gvxr.setDensity("cuboid", Ti90Al10, "g/cm3")

In [ ]:
#visualise()

In [ ]:
# Let's get an x-ray image
fig = plt.figure()
plt.imshow(gvxr.computeXRayImage(), cmap="gray")
plt.colorbar()
plt.show()

In [ ]:
def getXrayImage(x, take_screenshot=False):
    
    backup = gvxr.getLocalTransformationMatrix("root")

    # Move source, det, object using x
    x_src = x[0]
    y_src = x[1]
    z_src = x[2]
    gvxr.setSourcePosition(x_src, y_src, z_src, "mm")
    
    x_det = x[3]
    y_det = x[4]
    z_det = x[5]
    gvxr.setDetectorPosition(x_det, y_det, z_det, "mm")

    x_obj = x[6]
    y_obj = x[7]
    z_obj = x[8]

    alpha_x = x[9]
    alpha_y = x[10]
    alpha_z = x[11]

    gvxr.resetSceneTransformation();

    
    
#     gvxr.rotateNode("root", rot_angle, 1, 0, 0)
    
#     gvxr.translateNode("root", x_rot_axis_pos, y_rot_axis_pos, z_rot_axis_pos, "mm")

    
    
    gvxr.translateNode("root", x_obj, y_obj, z_obj, "mm")
    
    gvxr.rotateNode("root", alpha_x, 1, 0, 0)
    gvxr.rotateNode("root", alpha_y, 0, 1, 0)
    gvxr.rotateNode("root", alpha_z, 0, 0, 1)
    
    gvxr.translateNode("root", -x_obj, -y_obj, -z_obj, "mm")
    
    test_image = np.array(gvxr.computeXRayImage(), dtype=np.single)

    if take_screenshot:
        global screenshot
        gvxr.displayScene()        
        screenshot = gvxr.takeScreenshot()
    
    gvxr.setLocalTransformationMatrix("root", backup)
    
    return test_image / gvxr.getTotalEnergyWithDetectorResponse()

In [ ]:
import cma
from tifffile import imread, imwrite

In [ ]:
total_nb_projs = 200
angular_span = 360
angular_step = 360 / 103

fnames_images = []
angles = []

for i in range(total_nb_projs):
    angles.append(i*angular_step)
    fnames_images.append(current_folder + "/4thOctober_image/raw_images/raw_image_" + str(i) + ".tiff")

  

In [ ]:
def setNumberOfProjections(n):
    global nb_projs_in_fitness, ref_images, crop_ref_images, fitness_angles, fitness_fnames, test_images

    nb_projs_in_fitness = n
    
    ref_images = []
    fitness_angles = []
    fitness_fnames = []

    for i in range(nb_projs_in_fitness):
        index = int(i / nb_projs_in_fitness * total_nb_projs / 2)
        print(i, angles[index], fnames_images[index])
        ref_images.append(imread(fnames_images[index]))
        fitness_angles.append(angles[index])
        fitness_fnames.append(fnames_images[index])

    ref_images = np.array(ref_images, dtype=np.single)
    ref_images -= ref_images.mean()
    ref_images /= ref_images.std()

    fitness_angles = np.array(fitness_angles, dtype=np.single)

    test_images = np.zeros(ref_images.shape, dtype=np.single)
    
    crop_ref_images = ref_images[:, 211:470, 100:600]
    crop_ref_images -= crop_ref_images.mean()
    crop_ref_images /= crop_ref_images.std()

In [ ]:
def displayResult(x):
    transform(x)
    
    ZNCC = 100 * (crop_ref_images * crop_test_images).mean()
    print("Overall ZNCC:", "{:.4f}".format(ZNCC) + "%")
    
    for i, (angle, fname) in enumerate(zip(fitness_angles, fitness_fnames)):
        
        ref = np.copy(crop_ref_images[i])
        test = np.copy(crop_test_images[i])

        ref -= ref.mean()
        ref /= ref.std()
        
        test -= test.mean()
        test /= test.std()
        
        ZNCC = 100 * (ref * test).mean()

        offset=1.5
        # imwrite(current_folder + "/test_" + str(i) + "tif", test_images[i])
        fig, axs = plt.subplots(1, 3, figsize=(10, 4))
        plt.suptitle(fname + ", ZNCC=" + "{:.4f}".format(ZNCC) + "%")
        axs[0].imshow(ref_images[i],cmap="gray", vmin=-offset, vmax=offset)
        axs[1].imshow(test_images[i],cmap="gray", vmin=-offset, vmax=offset)
        # axs[2].imshow(np.abs(ref_images[i] - test_images[i]),cmap="gray")
        im = axs[2].imshow((ref_images[i] - test_images[i]),cmap="gray", vmin=-offset, vmax=offset)
        cbar = fig.colorbar(im)
        
        for ax in axs:
            ax.set_xlim([100, 600])
            ax.set_ylim([211, 470])

        fig = plt.figure(figsize=(10, 4))
        plt.plot(crop_ref_images[i][crop_ref_images.shape[1] // 2, :], label="Experimental")
        plt.plot(crop_test_images[i][crop_ref_images.shape[1] // 2, :], label="Simulated")
        plt.legend()
        plt.show()    

In [ ]:
setNumberOfProjections(2)